In [6]:
using Pkg;
Pkg.activate("..");

using Crystalline, MPBUtils
using PhotonicBandConnectivity, SymmetryBases
using TETB

  Activating project at `~/phd/TETB.jl`
┌ Warning: PyNormaliz could not be imported: related functionality of SymmetryBases.jl is nonfunctional
└ @ SymmetryBases ~/.julia/packages/SymmetryBases/fpjO6/src/SymmetryBases.jl:26


In [7]:
R1 = 0.2 #cylinder radius
mat = mp.Medium(epsilon=12)
geometry = [
    mp.Cylinder(radius=R1, center=[0, 0, 0], axis=[0, 0, 1], height=1, material=mat),
    mp.Cylinder(radius=R1, center=[0, 0, 0], axis=[0, 1, 0], height=1, material=mat),
    mp.Cylinder(radius=R1, center=[0, 0, 0], axis=[1, 0, 0], height=1, material=mat),
]
ms = mpb.ModeSolver(
    num_bands=8,
    geometry_lattice=mp.Lattice(basis1=[1, 0, 0], basis2=[0, 1, 0], basis3=[0, 0, 1],
        size=[1, 1, 1]),
    geometry=geometry,
    resolution=32,
)
ms.init_params(p=mp.ALL, reset_fields=true)

### obtain the symmetry vectors of the bands under study
sg_num = 221
band_summaries = obtain_symmetry_vectors(ms, sg_num)

vᵀ = band_summaries[1] # pick the 2 lower bands

Working in 3 dimensions.
Grid size is 32 x 32 x 32.
Solving for 8 bands at a time.
Creating Maxwell data...
Mesh size is 3.
Lattice vectors:
     (1, 0, 0)
     (0, 1, 0)
     (0, 0, 1)
Cell volume = 1
Reciprocal lattice vectors (/ 2 pi):
     (1, -0, 0)
     (-0, 1, -0)
     (0, -0, 1)
Geometric objects:
     cylinder, center = (0,0,0)
          radius 0.2, height 1, axis (0, 0, 1)
     cylinder, center = (0,0,0)
          radius 0.2, height 1, axis (0, 1, 0)
     cylinder, center = (0,0,0)
          radius 0.2, height 1, axis (1, 0, 0)
Geometric object tree has depth 7 and 15 object nodes (vs. 3 actual objects)
Initializing epsilon function...
Allocating fields...
Solving for band polarization: .
Initializing fields to random numbers...
solve_kpoint (0.5,0.5,0):
freqs:, k index, k1, k2, k3, kmag/2pi, band 1, band 2, band 3, band 4, band 5, band 6, band 7, band 8
Solving for bands 1 to 8...
    near maximum in trace
    linmin: converged after 7 iterations.
    iteration    1: trace =

2-band BandSummary:
 bands:      1:2
 n:          -Γ₁⁺+Γ₄⁻, R₃⁺, M₂⁺+M₃⁻, X₅⁻
 topology:   fragile

In [8]:
t = 1
brs = bandreps(sg_num)
d = matrix(brs)[end, :]

long_modes = find_auxiliary_modes(t, d, brs)

band_repre = find_all_band_representations(vᵀ, long_modes, d, brs)

nᵀ⁺ᴸ = brs[band_repre[2][1][1]...]
nᴸ = brs[band_repre[2][2]...]

println("nᵀ⁺ᴸ", " = ", nᵀ⁺ᴸ.label, " at ", nᵀ⁺ᴸ.wyckpos, "; nᴸ", " = ", nᴸ.label, " at ", nᴸ.wyckpos)

nᵀ⁺ᴸ = A₂g↑G at 3c; nᴸ = A₁ᵤ↑G at 1b


In [9]:
sgnum = 221
lgirs = realify(lgirreps(sgnum)["Γ"])
nfixed, nfree = physical_zero_frequency_gamma_irreps(
    lgirs;
    supergroup_constraints=true,
    force_fixed=true,
    lattice_reduce=true)
prettyprint_irrep_solspace(stdout, nfixed, nfree, lgirs)

-Γ₁⁺+Γ₄⁻ + (Γ₂⁺+Γ₅⁺-Γ₂⁻-Γ₅⁻)l + (Γ₁⁺-Γ₂⁺+Γ₄⁺-Γ₅⁺-Γ₁⁻+Γ₂⁻-Γ₄⁻+Γ₅⁻)p + (-Γ₁⁺-Γ₂⁺+Γ₃⁺+Γ₁⁻+Γ₂⁻-Γ₃⁻)q

In [11]:
brs´ = TETB.pick_klab_irreps_brs(brs, "Γ")
nᵀ⁺ᴸ´ = brs´[band_repre[2][1][1]...]
nᴸ´ = brs´[band_repre[2][2]...]

nᵀ´ = nᵀ⁺ᴸ´ - nᴸ´
vᵀ´ = TETB.pick_klab_irreps_vecs(vᵀ, "Γ")
nᵀ´ - vᵀ´.n

11-element Vector{Int64}:
  1
 -1
  0
  0
  0
  0
  1
 -1
  0
  0
  0

In [ ]:
Q⁻¹ * (nᵀ´-vᵀ´.n)[1:end-1]